In [ ]:
# if on google collab
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/Lol/models')
import c2v

data_path = '/content/drive/My Drive/Lol/data'
model_path = '/content/drive/My Drive/Lol/models'
c2v_name = 'c2v4'

In [ ]:
params_generator = {
    'batch_size': 1024,
    'shuffle': True,
}

experiment1 = c2v.Champ2Vec(
     c2v_name,
     model_path,
     data_path,
     params_generator
)

In [ ]:
experiment1.composition_counter()

In [ ]:
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers import BatchNormalization, LeakyReLU, Input,ReLU
from keras.optimizers import Adamax,SGD
from keras.models import Sequential, Model
import tensorflow as tf

In [ ]:
# the model
intermediate_layer_size = 4
input_team = Input(shape = (experiment1.N_champs,), name='input')
encoding = Dense(intermediate_layer_size, activation='tanh')(input_team) # , activation='tanh'
end = Dense(experiment1.N_champs, activation='softmax')(encoding)
# model is to train, encoder to use the first half of the model
model = Model(input_team, end)
encoder = Model(input_team, encoding)

In [ ]:
lr = 0.0003
optimizer=adamax = Adamax(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
loss = tf.keras.losses.CategoricalCrossentropy(
    from_logits=False, label_smoothing=0.7, reduction="auto", name="binary_crossentropy"
)
topKclass = tf.keras.metrics.TopKCategoricalAccuracy(
    k=5, name="top_k_categorical_accuracy", dtype=None
)

model.compile(optimizer=adamax, loss=loss, metrics=['accuracy', topKclass])

In [ ]:
experiment1.add_model(model, encoder)

In [ ]:
experiment1.train(400)

In [ ]:
experiment1.show_preds()

In [ ]:
experiment1.encoding_analysis()